In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import librosa 
from dataset import WavDataset
from config import *
import matplotlib.pyplot as plt

import h5py
from pathlib import Path
from config import *
import pickle

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [2]:
hdf5_dataset = h5py.File(INTERMEDIATE / 'embeddings.hdf5', 'r')
recs = list(hdf5_dataset)
annotated = [rec for rec in recs if np.array(hdf5_dataset[rec]['Y']).sum() > 0]
hdf5_dataset[annotated[0]]['X'].shape

(1249, 1024)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from tqdm import tqdm

def reduce_dimensions(embeddings, n_pca_components=50, n_tsne_components=3):
    pca = PCA(n_components=n_pca_components, random_state=42)
    pca_result = pca.fit_transform(embeddings)
    tsne = TSNE(n_components=n_tsne_components, random_state=42)
    tsne_result = tsne.fit_transform(pca_result)
    return tsne_result

def visualize_embeddings(tsne_result, labels):
    plt.figure(figsize=(10, 7))
    scatter = plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=labels, cmap='tab10', alpha=0.7)
    plt.colorbar(scatter)

l, e = [], []
for i in tqdm(np.random.randint(1, 100, size=100)):
    rec = annotated[i]
    embeddings = hdf5_dataset[rec]['X']
    labels = hdf5_dataset[rec]['Y']
    mask = [l.any() for l in labels]
    l.append(labels[mask, :])
    e.append(embeddings[mask, :])

all_e = np.concatenate(e, axis=0)
all_l = np.concatenate(l, axis=0)
print(
    all_e.shape, all_l.shape
)

 37%|██████████████████████████████▎                                                   | 37/100 [00:10<00:17,  3.69it/s]


TypeError: boolean index did not match indexed array

In [ ]:
tsne_result = reduce_dimensions(all_e)
visualize_embeddings(tsne_result, all_l)
plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def cluster_analysis(embeddings, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(embeddings)
    silhouette_avg = silhouette_score(embeddings, cluster_labels)
    print(f'Silhouette Score: {silhouette_avg}')
    return cluster_labels

cluster_labels = cluster_analysis(all_e, num_clusters=4)